# Document Clustering and Topic Modeling

*In* this project, we use unsupervised learning models to cluster unlabeled documents into different groups, visualize the results and identify their latent topics/structures.

## Contents

* [Part 1: Load Data](#Part-1:-Load-Data)
* [Part 2: Tokenizing and Stemming](#Part-2:-Tokenizing-and-Stemming)
* [Part 3: TF-IDF](#Part-3:-TF-IDF)
* [Part 4: K-means clustering](#Part-4:-K-means-clustering)
* [Part 5: Topic Modeling - Latent Dirichlet Allocation](#Part-5:-Topic-Modeling---Latent-Dirichlet-Allocation)


# Part 0: Setup Google Drive Environment

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# https://drive.google.com/open?id=192JMR7SIqoa14vrs7Z9BXO3iK89pimJL
file = drive.CreateFile({'id':'192JMR7SIqoa14vrs7Z9BXO3iK89pimJL'}) # replace the id with id of file you want to access
file.GetContentFile('data.tsv')  

# Part 1: Load Data

In [3]:
import numpy as np
import pandas as pd
import nltk

import gensim
# REGULAR EXPRESSION
import re

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# Load data into dataframe
df = pd.read_csv('data.tsv', sep='\t', header=0, error_bad_lines=False)

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

In [5]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [0]:
# Remove missing value
df.review_body.dropna(inplace=True)

In [0]:
# use the first 1000 data as our training data
data = df.loc[:1000, 'review_body'].tolist()

# Part 2: Tokenizing and Stemming

Load stopwords and stemmer function from NLTK library.
Stop words are words like "a", "the", or "in" which don't convey significant meaning.
Stemming is the process of breaking a word down into its root.

In [8]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

We use 179 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [0]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    # exclude stop words and tokenize the document, generate a list of string 
    tokens = [word.lower() for word in nltk.word_tokenize(text) if word not in stopwords]

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

# tokenization without stemming
def tokenization(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word not in stopwords]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [10]:
# tokenization and stemming
tokenization_and_stemming(data[0])

['absolut',
 'love',
 'watch',
 'get',
 'compliment',
 'almost',
 'everi',
 'time',
 'i',
 'wear',
 'dainti']

Use our defined functions to analyze (i.e. tokenize, stem) our reviews.

In [0]:
# 1. do tokenization and stemming for all the documents
# 2. also just do tokenization for all the documents
# the goal is to create a mapping from stemmed words to original tokenized words for result interpretation.
docs_stemmed = []
docs_tokenized = []
for i in data:
    tokenized_and_stemmed_results = tokenization_and_stemming(i)
    docs_stemmed.extend(tokenized_and_stemmed_results)
    
    tokenized_results = tokenization(i)
    docs_tokenized.extend(tokenized_results)

In [0]:
# create a mapping from stemmed words to original words
vocab_frame_dict = {docs_stemmed[x]:docs_tokenized[x] for x in range(len(docs_stemmed))}

# Part 3: TF-IDF

TF: Term Frequency

IDF: Inverse Document Frequency



In [13]:
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In total, there are 1000 reviews and 245 terms.


In [14]:
# check the parameters
tfidf_model.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 0.99,
 'max_features': 1000,
 'min_df': 0.01,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': 'english',
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <function __main__.tokenization_and_stemming>,
 'use_idf': True,
 'vocabulary': None}

Save the terms identified by TF-IDF.

In [0]:
# words
tf_selected_words = tfidf_model.get_feature_names()

In [16]:
# print out words
tf_selected_words

["'m",
 "'s",
 'abl',
 'absolut',
 'accur',
 'actual',
 'adjust',
 'alarm',
 'alreadi',
 'alway',
 'amaz',
 'amazon',
 'anoth',
 'arm',
 'arriv',
 'automat',
 'awesom',
 'bad',
 'band',
 'batteri',
 'beauti',
 'best',
 'better',
 'big',
 'bit',
 'black',
 'blue',
 'bought',
 'box',
 'br',
 'bracelet',
 'brand',
 'break',
 'bright',
 'broke',
 'button',
 'buy',
 'ca',
 'came',
 'case',
 'casio',
 'chang',
 'cheap',
 'clasp',
 'classi',
 'clock',
 'color',
 'come',
 'comfort',
 'compliment',
 'cool',
 'cost',
 'crown',
 'crystal',
 'dark',
 'date',
 'daughter',
 'day',
 'deal',
 'definit',
 'deliveri',
 'design',
 'dial',
 'differ',
 'difficult',
 'disappoint',
 'display',
 'dress',
 'durabl',
 'easi',
 'easili',
 'end',
 'everi',
 'everyday',
 'everyth',
 'exact',
 'excel',
 'expect',
 'expens',
 'face',
 'fair',
 'far',
 'fast',
 'featur',
 'feel',
 'fell',
 'fine',
 'finish',
 'fit',
 'function',
 'gave',
 'gift',
 'gold',
 'good',
 'got',
 'great',
 'hand',
 'happi',
 'hard',
 'heavi

# Part 4: K-means clustering

In [0]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=5)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [18]:
tfidf_matrix

<1000x245 sparse matrix of type '<class 'numpy.float64'>'
	with 7777 stored elements in Compressed Sparse Row format>

## 4.1. Analyze K-means Result

In [0]:
# create DataFrame films from all of the input files.
product = { 'review': df[:1000].product_title, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])

In [20]:
frame.head(10)

,review,cluster
0,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",3
1,Kenneth Cole New York Women's KC4944 Automatic...,3
2,Ritche 22mm Black Stainless Steel Bracelet Wat...,1
3,Citizen Men's BM8180-03E Eco-Drive Stainless S...,1
4,Orient ER27009B Men's Symphony Automatic Stain...,1
5,Casio Men's GW-9400BJ-1JF G-Shock Master of G ...,3
6,Fossil Women's ES3851 Urban Traveler Multifunc...,2
7,INFANTRY Mens Night Vision Analog Quartz Wrist...,1
8,G-Shock Men's Grey Sport Watch,2
9,Heiden Quad Watch Winder in Black Leather,1


In [21]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
1,660
3,102
2,99
4,76
0,63


In [22]:
km.cluster_centers_

array([[0.        , 0.00881011, 0.        , ..., 0.        , 0.00755492,
        0.        ],
       [0.0142842 , 0.05344821, 0.00526886, ..., 0.00627404, 0.01690282,
        0.01246096],
       [0.00171607, 0.00863063, 0.00318382, ..., 0.00213778, 0.00375703,
        0.02279456],
       [0.        , 0.01714556, 0.        , ..., 0.01232008, 0.01429803,
        0.00428619],
       [0.        , 0.03923788, 0.        , ..., 0.        , 0.00818984,
        0.        ]])

In [23]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(vocab_frame_dict[tf_selected_words[ind]])
        print (vocab_frame_dict[tf_selected_words[ind]] + ",", end='')
    print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
    print (", ".join(cluster_reviews))
    print ()

<Document clustering result by K-means>
Cluster 0 words:nice,watch,price,looks,really,good,
Cluster 0 reviews (63 reviews): 
Guess U13630G1 Men's day and date Gunmetal dial Gunmetal tone bracelet, Casio Men's MRW200H-7EV Sport Resin Watch, Casio G-shock Mudman Multiband6 Japanese Model [ Gw-9300-1jf ], Stuhrling Original Men's 395.33U16 Aquadiver Regatta Analog Swiss Quartz Stainless Steel Link Bracelet Watch, Invicta Men's Bolt 12298, Invicta Men's 3332 Force Collection Stainless Steel Left-Handed Watch with Black Leather Band, Timex Ironman Rugged 30 Full-Size Watch, Nautica Men's A16577G BFD 105 Chrono Analog Display Analog Quartz Black Watch, Invicta Women's 16267 ANGEL Analog Display Japanese Quartz Grey Watch, U.S. Polo Assn. Classic Men's US4023 Watch with Black Leather Band, Men's Gold-Tone Casio World Time Stainless Steel Watch A500WGA-1, Daniel Wellington Women's 0506DW Classic Southhampton Stainless Steel Watch With Multi-Color Striped Band, Domire Fashion Accessories Trial 

# Part 5: Topic Modeling - Latent Dirichlet Allocation

In [0]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
# LDA requires integer values
tfidf_model_lda = CountVectorizer(max_df=0.99, max_features=500,
                                 min_df=0.01, stop_words='english',
                                 tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix_lda = tfidf_model_lda.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix_lda.shape[0]) + \
      " reviews and " + str(tfidf_matrix_lda.shape[1]) + " terms.")

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['afterward', 'alon', 'alreadi', 'alway', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becom', 'besid', 'cri', 'describ', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'otherwis', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In total, there are 1000 reviews and 245 terms.


In [26]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix_lda)
print(lda_output.shape)
print(lda_output)

(1000, 5)
[[0.4672653  0.45670851 0.0254453  0.02534857 0.02523232]
 [0.79552186 0.05102421 0.05179199 0.05082874 0.0508332 ]
 [0.2        0.2        0.2        0.2        0.2       ]
 ...
 [0.10000009 0.10019819 0.59943424 0.10000254 0.10036493]
 [0.05034158 0.05119023 0.79693124 0.05068782 0.05084913]
 [0.03470466 0.86371275 0.03344739 0.03392228 0.03421292]]


In [27]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 245)
[[ 18.64418112 115.28425235   3.78346713 ...   0.20646286   1.68544059
   10.15695537]
 [ 11.34457751  26.05875076   1.24067667 ...   3.81576802  17.81737043
   16.8375157 ]
 [  0.20133249   0.57084154   0.20132706 ...   0.20067464   0.20281913
    0.2021173 ]
 [  3.20903109  50.52706855  11.57264333 ...  13.5739011   39.17660898
    3.24119439]
 [ 15.60087778  37.55908679   0.20188582 ...   0.20319338  13.11776086
   26.56221724]]


In [28]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.47,0.46,0.03,0.03,0.03,0
Doc1,0.80,0.05,0.05,0.05,0.05,0
Doc2,0.20,0.20,0.20,0.20,0.20,0
Doc3,0.03,0.03,0.03,0.63,0.28,3
Doc4,0.01,0.32,0.13,0.01,0.54,4
Doc5,0.04,0.04,0.84,0.04,0.04,2
Doc6,0.03,0.03,0.89,0.03,0.03,2
Doc7,0.03,0.03,0.03,0.88,0.03,3
Doc8,0.01,0.95,0.01,0.01,0.01,1
Doc9,0.02,0.02,0.27,0.02,0.67,4


In [29]:
df_document_topic['topic'].value_counts().to_frame()

,topic
2,328
0,215
1,169
4,144
3,144


In [30]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model_lda.get_feature_names()
df_topic_words.index = topic_names

df_topic_words.head()

[[ 18.64418112 115.28425235   3.78346713 ...   0.20646286   1.68544059
   10.15695537]
 [ 11.34457751  26.05875076   1.24067667 ...   3.81576802  17.81737043
   16.8375157 ]
 [  0.20133249   0.57084154   0.20132706 ...   0.20067464   0.20281913
    0.2021173 ]
 [  3.20903109  50.52706855  11.57264333 ...  13.5739011   39.17660898
    3.24119439]
 [ 15.60087778  37.55908679   0.20188582 ...   0.20319338  13.11776086
   26.56221724]]


,'m,'s,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,amaz,amazon,anoth,arm,arriv,automat,awesom,bad,band,batteri,beauti,best,better,big,bit,black,blue,bought,box,br,bracelet,brand,break,bright,broke,button,buy,ca,came,case,...,star,start,stop,strap,sturdi,style,stylish,super,sure,surpris,swim,tell,thank,thing,think,thought,time,timex,tini,tri,turn,use,valu,ve,want,watch,water,way,wear,week,weight,went,wife,wind,wish,work,worn,worth,wrist,year
Topic0,18.644181,115.284252,3.783467,17.193776,2.498196,4.426628,17.112108,6.562896,0.204727,0.201024,1.234214,12.061457,0.896925,1.195036,2.859796,0.204933,0.203661,16.027654,0.918035,8.205817,18.758706,0.200320,0.203059,0.629609,3.167914,4.867377,4.399140,35.433044,7.686660,0.202089,11.801080,0.209697,2.187102,0.201865,0.202058,2.367237,0.207019,0.202867,4.599534,0.201015,...,8.245239,7.438796,5.794320,0.201410,1.913627,0.201808,1.685917,0.201419,4.045626,6.248325,9.635921,7.030063,3.979924,14.121951,5.479003,3.539645,79.149411,0.203015,4.907643,0.202084,3.565092,18.020499,0.202210,10.074194,7.257453,148.012429,15.113087,0.202210,46.582156,0.201806,0.201536,1.568366,7.474268,3.789986,5.691063,22.519781,1.273391,0.206463,1.685441,10.156955
Topic1,11.344578,26.058751,1.240677,0.200782,1.564965,5.200017,0.313635,0.202175,7.772329,3.392527,0.201868,0.202625,24.254726,3.381520,0.202233,10.014081,1.699026,1.745276,24.515480,0.202065,32.429918,4.440855,0.201697,27.016950,0.203736,9.011704,25.992145,3.014304,0.201409,0.201111,1.593878,6.598054,5.761090,1.472886,12.218767,2.547949,24.801689,26.192354,18.308700,0.202928,...,0.207804,1.319304,0.201441,0.202737,2.668556,25.687462,4.918627,0.201407,3.000371,0.200999,4.760836,10.072345,0.203301,1.365277,15.087134,9.347278,34.080727,2.312111,0.200007,0.200477,0.200374,0.202402,13.732906,2.573807,26.841857,192.218357,0.202088,0.787336,30.963130,1.591989,12.827626,3.226198,0.200877,0.202428,0.202442,8.493901,8.520977,3.815768,17.817370,16.837516
Topic2,0.201332,0.570842,0.201327,0.201862,0.201670,0.201235,0.201735,0.200271,2.237422,0.201163,16.519592,0.204358,0.200765,0.201477,0.202719,0.202010,0.205951,0.201091,0.202643,0.202288,19.131671,0.201805,10.393989,6.239725,0.201092,0.201111,0.200459,0.202877,0.202387,0.201049,0.200429,0.204174,1.088618,0.201135,0.200331,0.204394,4.674490,0.200930,0.203436,0.200762,...,0.201423,0.200994,0.203018,0.200899,0.202799,1.788067,0.204247,11.520793,0.206731,0.218554,0.200670,0.201584,15.151435,0.202738,0.201736,0.200467,0.551957,0.200562,0.200870,0.201032,0.200012,8.232518,0.212262,0.200654,0.203915,169.523327,0.201326,2.162252,6.200834,1.842193,0.201246,0.200753,12.919853,0.200608,0.202004,30.105968,0.200762,0.200675,0.202819,0.202117
Topic3,3.209031,50.527069,11.572643,0.200819,0.203586,6.969778,7.170927,9.833238,0.202986,0.204504,2.048239,11.564682,2.387952,2.053578,14.767153,5.030656,22.691294,0.208752,58.548499,1.685029,7.399438,8.339967,8.710344,24.620305,19.226044,6.558732,0.205833,0.213992,6.702003,52.865596,0.203365,6.770181,1.006626,0.200671,0.200613,3.849673,14.567588,0.201375,0.209034,29.191457,...,6.571328,0.201416,1.297794,44.368651,2.700993,3.109757,12.986887,3.876372,9.397370,0.202888,0.201954,1.698836,0.207678,4.879505,7.989814,2.834294,26.531171,1.962094,2.917140,0.200629,9.833465,13.314897,3.187786,12.772345,0.203009,230.493020,0.201958,6.712515,10.242574,0.202829,0.204018,0.206655,0.203010,2.018787,4.449341,3.628806,3.801819,13.573901,39.176609,3.241194
Topic4,15.600878,37.559087,0.201886,0.202761,8.531583,0.202343,0.201594,0.201421,2.582536,10.000781,1.996087,3.966878,0.259633,9.168389,3.968100,2.548320,0.200068,7.817227,103.815341,37.704801,2.280267,2.817052,10.490912,2.493410,0.201215,5.361075,0.202424,11.135783,0.207541,196.530155,0.201249,0.217894,4.956564,8.923443,12.178232,8.030747,16.749214,0.202473,9.679296,0.203839,...,15.774206,6.839490,21.503426,3.026303,4.514024,0.212906,0.204322,0.200008,5.349902,6.129234,0.200619,3.997172,2.457662,4.430529,8.242313,1.078316,48.686733,17.322217,

In [31]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model_lda, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,watch,'s,love,time,like,wear,bought,n't,pretti,purchas,month,disappoint,set,face,work
Topic 1,watch,n't,look,color,great,face,light,time,realli,beauti,wear,love,big,want,ca
Topic 2,watch,good,nice,great,love,look,perfect,price,qualiti,excel,work,product,far,gift,pictur
Topic 3,watch,look,band,size,br,'s,like,strap,wrist,price,great,qualiti,small,hand,case
Topic 4,watch,br,band,n't,work,time,week,day,use,replac,batteri,'s,look,good,link
